In [2]:
from hydra import initialize, compose
import numpy as np
import polars as pl
from pathlib import Path
import json
from src.utils import construct_sparse_adj_mat, load_json, augment_idx_feature

In [3]:
with initialize(config_path="../configs/filepaths", version_base=None):
    fps = compose(config_name='base')

In [4]:
dataset = "sprhea"
toc = "v3_folded_pt_ns"

In [9]:
def load_data(dataset, toc, negative_sampling):
    # Load reaction data
    _rxns = load_json(Path(fps.data) / dataset / f"{toc}.json")
    adj, idx_sample, idx_feature = construct_sparse_adj_mat(
        Path(fps.data) / dataset / (toc + ".csv")
    )
    X_pos = list(zip(*adj.nonzero()))

    if negative_sampling == 'alternate_reaction_center':
        unobs_rxns = load_json(Path(fps.data) / dataset / f"{toc}_arc_unobserved_reactions.json")
        reactions = {**_rxns, **unobs_rxns}
        idx_feature = augment_idx_feature(idx_feature, unobs_rxns)

    else:
        reactions = _rxns

    return reactions, idx_feature

In [10]:
random_rxns, random_idx_ft = load_data(dataset, toc, negative_sampling='random')
arc_rxns, arc_idx_ft = load_data(dataset, toc, negative_sampling='alternate_reaction_center')

Constructing v3_folded_pt_ns sparse adjacency matrix
Constructing v3_folded_pt_ns sparse adjacency matrix


In [11]:
print(len(random_idx_ft), len(arc_idx_ft))

6460 76292


In [12]:
rxnfp_vecs = np.load(
    "/home/stef/quest_data/hiec/data/pretrained_rxn_embeddings/sprhea_v3_folded_pt_ns_rxnfp.npy"
)
print(rxnfp_vecs.shape)

(76292, 256)
